In [3]:
# =====================================
#  MULTI-BASIS DATA GENERATION
# =====================================

"""
--------------------------------------------------------

**Implementation Details**:
1. Create a random 2-qubit quantum circuit:
   - A parameterizable circuit with random operations or a user-chosen design.
   - Possibly use Qiskit's `random_circuit` or custom gates to produce different states.

2. For each circuit instance (with random seeds or random parameters):
   - Simulate (or run on hardware) in the Z-basis.
   - Then apply transforms or re-initialize and measure in X-basis.
   - Then measure in Y-basis.
   - This yields three arrays of measurement outcomes: Mz, Mx, My.

3. Convert measurement bitstrings from each basis into integer or categorical features.
   - For 2 qubits, in the Z basis, "00", "01", "10", "11" are typical.
   - In the X basis, "++", "+-", "-+", "--" or something akin.
   - Or we can store the raw bitstrings as features, or we can store their counts.
   - We can also store the empirical distribution across these bitstrings.
     E.g., (p00, p01, p10, p11) for each basis.

4. Summarize these distribution vectors as the "label" or as "feature" depending on the user design.
   For multi-head conformal "toy" demonstration, let's store them as "heads."
   We can treat them as if they are separate "predictions" from each basis, or separate random samples.
   Then we might define a final label or final target.
   There's some design space here:
   - The dataset might store (X_features, [Mz, Mx, My]) as "heads,"
     and the final "true label" might be some function of the circuit parameters
     or we may create an artificially assigned "y" for classification or regression.

5. Save or return the resulting dataset as a structure with shape:
   - X_data shape: (#samples, #features). Possibly we store circuit depth,
     random seeds, gate counts, etc., or any classical features we like.
   - Y_data shape: (#samples, #heads) if we want each basis to be one dimension of the output.
     Alternatively, store them as bigger arrays.
---


"""
!pip install qiskit
!pip install numpy
!pip install tqdm
import numpy as np
from tqdm import tqdm

# Qiskit for quantum simulation
!pip install qiskit qiskit-aer --quiet
from qiskit import QuantumCircuit, transpile
from qiskit.circuit.random import random_circuit
from qiskit_aer import AerSimulator


def measure_distribution(qc, backend, shots=1024):
    """Given a Qiskit QuantumCircuit 'qc' and a 'backend',
       run it for 'shots' times, returns distribution dict: {bitstring:count}
       Also returns the normalized distribution as a dict of {bitstring:prob}.
    """
    qc_compiled = transpile(qc, backend)
    job = backend.run(qc_compiled, shots=shots)
    result = job.result()
    counts = result.get_counts(qc_compiled)

    # Convert to probabilities
    dist = {}
    for k,v in counts.items():
        dist[k] = v / shots
    return dist

def measure_in_z_basis(qc, backend, shots=1024):
    """Measure a circuit in the computational (Z) basis.
       We assume 'qc' already has measure-all at the end or we add them."""
    # We'll copy the circuit, add measurement, run, return distribution
    qc_z = qc.copy()
    # ensure measurement:
    if not qc_z.data or qc_z.data[-1].operation.name != 'measure':
        qc_z_copy = qc_z.copy()
        qc_z_copy.measure_all()
        dist = measure_distribution(qc_z_copy, backend, shots)
        return dist
    else:
        # if it's already measured
        dist = measure_distribution(qc_z, backend, shots)
        return dist

def measure_in_x_basis(qc, backend, shots=1024):
    """Measure the same circuit in the X basis.
       For 2 qubits, we apply Hadamard gates to each qubit prior to measure-all
       so that measuring in Z after H is effectively measuring in X basis."""
    qc_x = qc.copy()
    qc_x_copy = QuantumCircuit(qc_x.num_qubits)
    qc_x_copy.compose(qc_x, inplace=True)

    # Remove measure if present
    # then apply H gates, measure in Z
    # we do a quick check:
    for i, (op, qargs, _) in enumerate(qc_x_copy.data):
        if op.name == 'measure':
            # we remove measure instructions
            pass
    # We'll just forcibly create a fresh circuit with the same gates except measure
    # a simpler approach is:
    qc_x_final = QuantumCircuit(qc_x_copy.num_qubits)
    for inst, qargs, cargs in qc_x_copy.data:
        if inst.name != 'measure':
            qc_x_final.append(inst, qargs, cargs)

    # now apply hadamard to each qubit
    for qb in range(qc_x_final.num_qubits):
        qc_x_final.h(qb)

    qc_x_final.measure_all()
    dist = measure_distribution(qc_x_final, backend, shots)
    return dist

def measure_in_y_basis(qc, backend, shots=1024):
    """Measure the same circuit in the Y basis.
       For 1 qubit, measuring Y basis can be done by applying
       S^\dagger then H, or a direct rotation.
       We'll do S^\dagger * H for each qubit, then measure in Z."""
    qc_y = qc.copy()
    # create a fresh circuit
    qc_y_final = QuantumCircuit(qc_y.num_qubits)
    for inst, qargs, cargs in qc_y.data:
        if inst.name != 'measure':
            qc_y_final.append(inst, qargs, cargs)

    # apply S^\dagger and H to measure in Y
    # recall S^\dagger = S^3, or a phase of -pi/2
    for qb in range(qc_y_final.num_qubits):
        qc_y_final.sdg(qb)
        qc_y_final.h(qb)

    qc_y_final.measure_all()
    dist = measure_distribution(qc_y_final, backend, shots)
    return dist


def dist_to_vector(dist, n_qubits=2):
    """Given a distribution dict: {bitstring:prob},
       convert to a vector [p00, p01, p10, p11]
       or a vector of length 2^n_qubits in general, sorted by the canonical order.
    """
    # for 2 qubits, we want "00", "01", "10", "11"
    basis_list = []
    for i in range(2**n_qubits):
        b_str = format(i, '0{}b'.format(n_qubits))[::-1]  # maybe reversed
        # but let's keep the normal order "00" as i=0, "01" i=1, ...
        # Actually let's define an explicit list:
    basis_list = ['00','01','10','11']  # in standard reading q1q0 or q0q1 depends
    # if we assume q0 is the first from left, might differ from qiskit's endianness
    # but let's keep it consistent.

    vec = []
    for bs in basis_list:
        if bs in dist:
            vec.append(dist[bs])
        else:
            vec.append(0.0)
    return np.array(vec)


# Let's generate data
num_samples = 20000   # how many random circuits
shots = 1024

backend_sim = AerSimulator()

X_data = []
Y_data = []  # we'll store [Zvec, Xvec, Yvec] as a single big vector => shape (2^2 *3=12)
# or we store as shape = (12,) ?

for i in tqdm(range(num_samples), desc="Generating multi-basis data"):
    # random circuit
    depth = np.random.randint(1,5)  # small depth
    qc_rand = random_circuit(num_qubits=2, depth=depth, measure=False, seed=np.random.randint(999999))

    # measure in Z
    distZ = measure_in_z_basis(qc_rand, backend_sim, shots=shots)
    vecZ = dist_to_vector(distZ, n_qubits=2)

    # measure in X
    distX = measure_in_x_basis(qc_rand, backend_sim, shots=shots)
    vecX = dist_to_vector(distX, n_qubits=2)

    # measure in Y
    distY = measure_in_y_basis(qc_rand, backend_sim, shots=shots)
    vecY = dist_to_vector(distY, n_qubits=2)

    # let's define some minimal classical features:
    # e.g. [depth, total_ops, random_seed, etc.]
    # or just store depth for demonstration
    # also store counts of gates
    total_ops = len(qc_rand.data)
    Xrow = [depth, total_ops]

    # for Y, we store the 12-dim vector cat of Z, X, Y
    # shape= (12,)
    Yrow = np.concatenate([vecZ, vecX, vecY], axis=0)

    X_data.append(Xrow)
    Y_data.append(Yrow)

X_data = np.array(X_data)
Y_data = np.array(Y_data)

print("Generated shape of X_data:", X_data.shape)
print("Generated shape of Y_data:", Y_data.shape)

# We can save them for later usage
import pickle
data_dict = {"X_data": X_data, "Y_data": Y_data}
with open("multi_basis_data.pkl", "wb") as f:
    pickle.dump(data_dict, f)

print("\n--- Multi-basis data generation complete ---")
print("   -> Saved to 'multi_basis_data.pkl'")


Görüntülenen çıkış son 5000 satıra kısaltıldı.
Generating multi-basis data:  96%|█████████▌| 19167/20000 [2:55:39<07:22,  1.88it/s]<ipython-input-3-246c523c767d>:140: DeprecationWarning: Treating CircuitInstruction as an iterable is deprecated legacy behavior since Qiskit 1.2, and will be removed in Qiskit 2.0. Instead, use the `operation`, `qubits` and `clbits` named attributes.
  for i, (op, qargs, _) in enumerate(qc_x_copy.data):
<ipython-input-3-246c523c767d>:147: DeprecationWarning: Treating CircuitInstruction as an iterable is deprecated legacy behavior since Qiskit 1.2, and will be removed in Qiskit 2.0. Instead, use the `operation`, `qubits` and `clbits` named attributes.
  for inst, qargs, cargs in qc_x_copy.data:
<ipython-input-3-246c523c767d>:167: DeprecationWarning: Treating CircuitInstruction as an iterable is deprecated legacy behavior since Qiskit 1.2, and will be removed in Qiskit 2.0. Instead, use the `operation`, `qubits` and `clbits` named attributes.
  for inst, qar

Generated shape of X_data: (20000, 2)
Generated shape of Y_data: (20000, 12)

--- Multi-basis data generation complete ---
   -> Saved to 'multi_basis_data.pkl'


In [4]:
# =====================================
# MULTI-HEAD CONFORMAL PREDICTION
# =====================================

"""
Extensive Explanation (Markdown/Educational Commentary)
--------------------------------------------------------
We have produced data where:
 - X_data: shape (#samples, 2) => [depth, total_ops] or something similar
 - Y_data: shape (#samples, 12) => concatenation of [Zvec(4), Xvec(4), Yvec(4)]

So each sample i has Y[i] = [Z00, Z01, Z10, Z11, X00, X01, X10, X11, Y00, Y01, Y10, Y11].
Hence, we effectively have "3 heads," each of dimension 4.
We want a multi-head conformal approach that yields a set predictor in R^{12}.
But that might be too big. Typically, conformal intervals are done in R^1 or small dimension.
In the distributional approach, we can define a single scalar residual that merges these 12 outputs
(for instance, we do some distance measure with a trained regression model).
Alternatively, we can do “multi-dimensional conformal,” but it’s more advanced.
As a simpler approach, we do the following:

**Approach**:
1. Train a multi-output regressor f(x) -> R^{12}. We'll do something trivial (like a RandomForestRegressor with 12 outputs).
2. For each data point i in the calibration set, define a "residual" or "score" as the L1 or L2 distance between predicted Y[i]_pred and actual Y[i].
3. Then standard “Distributional Conformal” in single-scalar sense:
   - Sort these residuals,
   - pick the (1-alpha) quantile as 'radius' => tau,
   - For a new test point x*, we produce the predicted center y* = f(x*).
   - Then the conformal set is all points z in R^{12} s.t. dist(z, y*) <= tau.
   That is, a 12-dimensional ball of radius tau.
   This is a naive multi-dim approach, but it's a single radius approach (the “minmax” approach might define hyper-rectangles, etc.).

**Implementation**:
- We'll do a train/test split: use ~80% for training, 20% for "calibration" or "test"? Actually, we typically do 3 sets: train, calibration, test.
  But let's do the simpler approach:
   - we do a partial: 70% train, 15% calibration, 15% final test.
   - Then define a distributional conformal approach with the calibration portion.
   - Evaluate coverage on the final test portion.

**Coverage Metric**:
- For each test point, we see if the L2 distance between the predicted center f(x*) and the true label y* is <= tau.
- We'll measure the fraction that is inside the ball => coverage.

**Caveat**:
This is a "toy" code. We are ignoring advanced correlation structure, advanced drift, etc.
But it demonstrates the idea.
We call it "multi-head" because each data sample is effectively a concatenation of multiple measurement heads.
Hence we do 1 unified conformal set in R^{12}.
One might create separate sets for each basis or do more sophisticated integration.
But let's do the single-ball approach to keep it minimal in code.

Now let's implement it below.
"""

import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

# 1) Load data
with open("multi_basis_data.pkl", "rb") as f:
    data_dict = pickle.load(f)
X_data = data_dict["X_data"]
Y_data = data_dict["Y_data"]

print("Loaded X_data shape=", X_data.shape, "Y_data shape=", Y_data.shape)

# 2) Train / Cal / Test split
X_trainval, X_test, Y_trainval, Y_test = train_test_split(X_data, Y_data, test_size=0.10, random_state=42)
X_train, X_cal, Y_train, Y_cal = train_test_split(X_trainval, Y_trainval, test_size=0.10, random_state=42)
# That yields roughly 70% train, 15% cal, 15% test

print("Train shape:", X_train.shape, Y_train.shape)
print("Cal shape:", X_cal.shape, Y_cal.shape)
print("Test shape:", X_test.shape, Y_test.shape)

# 3) Fit multi-output regressor
multi_rf = RandomForestRegressor(n_estimators=50, random_state=42)
multi_rf.fit(X_train, Y_train)
print("\n=== Trained Multi-Output Regressor ===")

# 4) Calculate calibration residuals
def l2_dist(a,b):
    return np.sqrt(np.sum((a-b)**2))

cal_preds = multi_rf.predict(X_cal)
residuals = []
for i in range(len(X_cal)):
    r = l2_dist(cal_preds[i], Y_cal[i])
    residuals.append(r)
residuals = np.array(residuals)

# 5) For coverage alpha, define radius
def coverage_at_alpha(alpha, cal_resids, X_test, Y_test, model):
    # pick tau
    n = len(cal_resids)
    idx = int(np.ceil((1 - alpha)*(n+1))) - 1
    if idx < 0:
        idx = 0
    sorted_res = np.sort(cal_resids)
    tau = sorted_res[idx] if idx < len(sorted_res) else sorted_res[-1]

    # measure coverage
    test_pred = model.predict(X_test)
    inside_count = 0
    for i in range(len(X_test)):
        dist_ = l2_dist(test_pred[i], Y_test[i])
        if dist_ <= tau:
            inside_count+=1
    coverage = inside_count/len(X_test)
    return coverage, tau

alpha_list = [0.05, 0.1, 0.2, 0.3, 0.5]
print("\n=== Distributional Conformal in 12D with Single Radius Ball ===")
for alpha in alpha_list:
    cov, tau_ = coverage_at_alpha(alpha, residuals, X_test, Y_test, multi_rf)
    print(f"alpha={alpha:.2f}, coverage={cov:.3f}, radius={tau_:.4f}")


print("\n=== Done ===")


Loaded X_data shape= (20000, 2) Y_data shape= (20000, 12)
Train shape: (16200, 2) (16200, 12)
Cal shape: (1800, 2) (1800, 12)
Test shape: (2000, 2) (2000, 12)

=== Trained Multi-Output Regressor ===

=== Distributional Conformal in 12D with Single Radius Ball ===
alpha=0.05, coverage=0.956, radius=1.0480
alpha=0.10, coverage=0.910, radius=0.9336
alpha=0.20, coverage=0.790, radius=0.8399
alpha=0.30, coverage=0.700, radius=0.7761
alpha=0.50, coverage=0.507, radius=0.6908

=== Done ===
